In [1]:
#             Подключение библиотек
# Отправка HTTP запросов и возврат ответов
import requests
# Обработка ответов в формате html
from bs4 import BeautifulSoup as bs
# Работа с .json
import json
# Создание логов
import logging

In [2]:
#             Конфигурация

# Папка проекта
folderForResults = 'korchagin'
# Имя лог-файла
logFileName = 'picture.log'
# Имя файла-результата
outFileName = 'result.json'

# URL ресурса
resourceURL = 'https://www.imdb.com/'
# URL страницы поиска
searchPageURL = 'https://www.imdb.com/search/title'

#     Фильтрация
filters = {
##     TITLE_TYPE
##     May be: feature,tv_movie,tv_series,tv_special,tv_miniseries,documentary,video_game,short,video,tv_short

    # 'title_type': "feature,tv_short",
    
##     RELEASE_DATE
##     Format: YYYY-MM-DD, YYYY-MM, or YYYY
##     Example: "2000-01-01,2013-12-31" or "2000-01-01," or ",2013-12-31"

    # 'release_date': "",
    
##     GENRES
##     May be:  Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western
    'genres': "Action,Adventure",
#     'genres': 'reality-tv,romance,talk-show,thriller',   
    
##     USER_RAITING
##     Example: "7.0,10.0" or "7.0," or ",5.0"
#     'user_rating': ""
    
##     COUNTRIES
##     Abbreviated name from the list at the end of the notebook
##     Example: "ax,ad"
#       'countries': "ru,gb,us",
    
##     COUNT
    'count': 250,
    'start': 1
}

# СКОЛЬКО КАРТИН ОБРАБОТАТЬ
desiredNumberOfImages = 1000
# обработано в настоящий момент
currentNumberOfPictures = 0

# Настройка логов
logging.basicConfig(handlers=[logging.FileHandler(logFileName, 'w', 'utf-8')], format='[%(levelname)s] : %(asctime)s : %(message)s', level=logging.INFO)

In [4]:
# 1. Отправляем запрос с фильтрами
try:
    # Результирующий массив
    completedPictures = []
    # Оборачиваем в цикл, чтобы обработать максимально разрешенные 250 картин
    # "Пока количество необх. картин < текущ. знач
    while desiredNumberOfImages > len(completedPictures):
        # Посылаем GET запрос, в ответ получаем html страничку с результатами поиска.
        searchPage = requests.get(searchPageURL, params=filters)
        logging.info("Запрос " + searchPage.url)
        print("Запрос " + searchPage.url + " [" + str(currentNumberOfPictures) + " / " + str(desiredNumberOfImages) + "]")
        if searchPage.status_code != 200:
            # Если запрос пришел не с кодом "без ошибок", то вызываем исключение.
            raise Exception("Запрос получения страницы поиска пришел с ошибкой. КОД " + str(searchPage.status_code) + ".")
        # Теперь необходимо распарсить эту страничку и вытянуть нужное нам.
        soup = bs(searchPage.text, 'html.parser')
        # Проверим, нашлось ли чего по нашему фильтру
        if soup.find(class_='desc').text.strip() == 'No results.':
            raise Exception("Результаты с такими параметрами фильтрации не найдены: " + json.dumps(filters))
        # Еще проверим, а не все ли фильмы мы уже посмотрели, то есть других то больше нет.
        if soup.find(class_='desc').text.strip() == str(len(completedPictures)) + " titles.":
            break
        logging.info("Объекты с параметрами фильтрации найдены.")
        # Найдем все тэги с классом "lister-item mode-advanced"
        listOfWorks = soup.find_all(class_='lister-item mode-advanced')
        # Так как элементов с классом "lister-item mode-advanced" много (ну или должно быть много)
        # необходимо "пройтись по каждому", используем ЦИКЛ.
        for product in listOfWorks:
            # Если мы уже сохранили достаточно картин, то выходим.
            if currentNumberOfPictures > desiredNumberOfImages:
                break
            # Теперь в каждом фильме нам нужно найти что-то конкретное, например НАЗВАНИЕ
            # В скриншотах второго сообщения будет инфа, почему я выбрал именно этот блок, а так же как я его нашел, собственно
            # И сразу маленькая деталь. Так как тег <a> (ссылка в html) идет без класса или другой идентификации 
            # (а таких ссылок даже у 1 фильма несколько)
            productInfo = product.find('h3', class_='lister-item-header')
            if productInfo is not None:
                # ИМЯ
                name = productInfo.find('a')
                
                # Переходим на страницу фильма для получения полной информации о нем
                # Формируем ссылку, используя атрибут href тега <a>
                logging.info("Сбор информации со страницы " + resourceURL + name.attrs["href"])
                page = requests.get(resourceURL + name.attrs["href"])
                logging.info("- Статус ответа: " + str(page.status_code)) # Если код 200 (см. протокол HTTP), то все ОК!
                if page.status_code != 200:
                    logging.critical("Запрос пришел с ошибкой. КОД " + str(page.status_code))
                    raise Exception("Запрос пришел с ошибкой. КОД " + str(page.status_code))
                soup = bs(page.text, 'html.parser')
                
                # ЖАНР
                # Блоков класса "see-more inline canwrap" может быть несколько. Нам нужен тот, в котором есть "Genres:"
                divs = soup.find_all(class_='see-more inline canwrap')
                genres = []
                for div in divs:
                    if div.find('h4', class_='inline') and div.find('h4', class_='inline').text == 'Genres:':
                        logging.info('- Жанр обнаружен')
                        # Жанры выделены в отдельные ссылки (т.е. теги <a>), 
                        # находим их все а дальше складываем в массик genres
                        # на всякий удаляем пробелы слева и справа от текста в ссылке
                        geners_list = div.find_all('a')
                        for genre in geners_list:
                            genres.append(genre.text.strip()) 
                            
                # РЕЙТИНГ
                
                ratings = soup.find_all(class_='ratings_wrapper')
                if len(ratings) > 0:
                    # Фильму простравлен рейтинг, находим оценку и записываем.
#                     logging.info("- Есть рейтинг") #UPD: НЕ ВСЕГДА www.imdb.com//title/tt8804038/
                    # Зная, что "ratings_wrapper" на странице один, а метод возвращает массив объектов,
                    # нам нужно взять самый первый и работать с ним
                    ratings = ratings[0]
                    # Ищем атрибут string, затем span, берем у последнего текст
                    if ratings.find('strong'):
                        ratings = float(ratings.find('strong').find('span').text)
                        logging.info("- Есть рейтинг")
                    else:
                        ratings = "Фильм еще не имеет оценок"
                        logging.info("- " + ratings)
                else:
                    # У фильма нет рейтинга. Вероятно, он еще не вышел или не был оценен.
                    ratings = "Фильм еще не имеет оценок"
                    logging.info("- " + ratings)
                    
                # Топ каста (stars)
                cast = ""
                if soup.find('table', class_='cast_list'):
                    cast = soup.find('table', class_='cast_list')
                    if cast:
                        # Если у фильма есть список актеров (гипотетически, он должен быть всегда)
                        logging.info("- Список актеров присутствует")
                        # Я знаю, что таблица "cast_list" состоит из строк. 
                        # Найдем их.
                        cast_rows = cast.find_all('tr')
                        cast = []

                        # Первая строка - информация, ее нужно пропустить
                        for i in range(len(cast_rows[1:])):
                            if cast_rows[i+1].find('a') and cast_rows[i+1].find('a').img.attrs["title"]:
                                cast.append(cast_rows[i+1].find('a').img.attrs["title"])
                    else:
                        cast = "Список актеров отсутствует"
                    
                # Тип (сериал, фильм и т.д.)
                type_film = "Фильм"
                if soup.find(class_='seasons-and-year-nav'):
                    # Ищем блок с классом, в котором располагаются ссылки на сезоны
                    typeFilmDivision = soup.find(class_='seasons-and-year-nav')
                    if typeFilmDivision.find('a'):
                        # Если мы можем вытянуть номер последнего сезона (это всегда первая ссылка)
                        # то прописываем его в скобках [N]
                        type_film = "Сериал [" + typeFilmDivision.find('a').text + "]"
                    else:
                        type_film = "Сериал"
                else: type_film = "Фильм"
                logging.info("- Это " + type_film)
                
                # Блоки Details, Box office, Technical specs
                Details = []
                # Находим div
                if soup.find(id='titleDetails'):
                    divisionDetails = soup.find(id='titleDetails')
                    textFieldsOfDetails = divisionDetails.find_all(class_='txt-block')
                    for textField in textFieldsOfDetails:
                        if textField.find('h4') and textField.find('a') and textField.find('a').text != 'See more':
                            headingOfDetail = textField.find('h4').text
                            textOfDetail = textField.find('a').text
                            if textField.find('a'):  # Вытягиваем ссылку на подробность
                                Details.append({headingOfDetail.replace(":", "") : textOfDetail, 'link': textField.find('a').attrs["href"]})
                            else:
                                Details.append({headingOfDetail.replace(":", "") : textOfDetail})
                        elif textField.find('h4'):
                            # Если у нас нет ссылки, то информация следует сразу за объявлением <div>.
                            # Для того, чтобы достать эту информацию, необходимо получить строки
                            # без пробелов и символов начала строки. В этом поможет метод "stripped_strings"
                            # Экмпериментально доказано, что таким образом нужную информацию можно выловаить, если взять 2 эл-т.
                            Details.append({textField.find('h4').text.replace(":", "") : list(textField.stripped_strings)[1]})
                
                #  Добавляем в массив результатов
                completedPictures.append({'name': name.text, 'ratings': ratings, 'cast': cast, 'genres': genres, 'type': type_film, 'details': Details})
                logging.info("ВЫПОЛНЕНО: " + str(name.text))
                currentNumberOfPictures += 1
        filters['start'] = len(completedPictures)   
    with open(outFileName, 'w', encoding='utf-8') as f:
        # Записываем в файл результат, конвертированный в json
        f.write(json.dumps(completedPictures, sort_keys=True, indent=4, ensure_ascii=False))
    logging.info("Информация записана в файл " + str(outFileName))
    logging.info("Записей обработано: " + str(len(completedPictures)) + "\n")
    print("DONE")
            
except Exception as e:
    logging.error(e)
    print(e)


Запрос https://www.imdb.com/search/title/?genres=Action%2CAdventure&count=250&start=1 [0 / 1000]
Запрос https://www.imdb.com/search/title/?genres=Action%2CAdventure&count=250&start=250 [250 / 1000]
Запрос https://www.imdb.com/search/title/?genres=Action%2CAdventure&count=250&start=500 [500 / 1000]
Запрос https://www.imdb.com/search/title/?genres=Action%2CAdventure&count=250&start=750 [750 / 1000]
DONE


LIST OF COUNTRIES AND ABBREVIATIONS
======

| KODE | COUNTRY
| :---: | :--- 
| af | Afghanistan |
| ax | Åland Islands |
| al | Albania |
| dz | Algeria |
| as | American Samoa |
| ad | Andorra |
| ao | Angola |
| ai | Anguilla |
| aq | Antarctica |
| ag | Antigua and Barbuda |
| ar | Argentina |
| am | Armenia |
| aw | Aruba |
| au | Australia |
| at | Austria |
| az | Azerbaijan |
| bs | Bahamas |
| bh | Bahrain |
| bd | Bangladesh |
| bb | Barbados |
| by | Belarus |
| be | Belgium |
| bz | Belize |
| bj | Benin |
| bm | Bermuda |
| bt | Bhutan |
| bo | Bolivia |
| bq | Bonaire, Sint Eustatius and Saba |
| ba | Bosnia and Herzegovina |
| bw | Botswana |
| bv | Bouvet Island |
| br | Brazil |
| io | British Indian Ocean Territory |
| vg | British Virgin Islands |
| bn | Brunei Darussalam |
| bg | Bulgaria |
| bf | Burkina Faso |
| bumm | Burma |
| bi | Burundi |
| kh | Cambodia |
| cm | Cameroon |
| ca | Canada |
| cv | Cape Verde |
| ky | Cayman Islands |
| cf | Central African Republic |
| td | Chad |
| cl | Chile |
| cn | China |
| cx | Christmas Island |
| cc | Cocos (Keeling) Islands |
| co | Colombia |
| km | Comoros |
| cg | Congo |
| ck | Cook Islands |
| cr | Costa Rica |
| ci | Côte d'Ivoire |
| hr | Croatia |
| cu | Cuba |
| cy | Cyprus |
| cz | Czech Republic |
| cshh | Czechoslovakia |
| cd | Democratic Republic of the Congo |
| dk | Denmark |
| dj | Djibouti |
| dm | Dominica |
| do | Dominican Republic |
| ddde | East Germany |
| ec | Ecuador |
| eg | Egypt |
| sv | El Salvador |
| gq | Equatorial Guinea |
| er | Eritrea |
| ee | Estonia |
| et | Ethiopia |
| fk | Falkland Islands |
| fo | Faroe Islands |
| yucs | Federal Republic of Yugoslavia |
| fm | Federated States of Micronesia |
| fj | Fiji |
| fi | Finland |
| fr | France |
| gf | French Guiana |
| pf | French Polynesia |
| tf | French Southern Territories |
| ga | Gabon |
| gm | Gambia |
| ge | Georgia |
| de | Germany |
| gh | Ghana |
| gi | Gibraltar |
| gr | Greece |
| gl | Greenland |
| gd | Grenada |
| gp | Guadeloupe |
| gu | Guam |
| gt | Guatemala |
| gg | Guernsey |
| gn | Guinea |
| gw | Guinea-Bissau |
| gy | Guyana |
| ht | Haiti |
| hm | Heard Island and McDonald Islands |
| va | Holy See (Vatican City State) |
| hn | Honduras |
| hk | Hong Kong |
| hu | Hungary |
| is | Iceland |
| in | India |
| id | Indonesia |
| ir | Iran |
| iq | Iraq |
| ie | Ireland |
| im | Isle of Man |
| il | Israel |
| it | Italy |
| jm | Jamaica |
| jp | Japan |
| je | Jersey |
| jo | Jordan |
| kz | Kazakhstan |
| ke | Kenya |
| ki | Kiribati |
| xko | Korea |
| xkv | Kosovo |
| kw | Kuwait |
| kg | Kyrgyzstan |
| la | Laos |
| lv | Latvia |
| lb | Lebanon |
| ls | Lesotho |
| lr | Liberia |
| ly | Libya |
| li | Liechtenstein |
| lt | Lithuania |
| lu | Luxembourg |
| mo | Macao |
| mg | Madagascar |
| mw | Malawi |
| my | Malaysia |
| mv | Maldives |
| ml | Mali |
| mt | Malta |
| mh | Marshall Islands |
| mq | Martinique |
| mr | Mauritania |
| mu | Mauritius |
| yt | Mayotte |
| mx | Mexico |
| md | Moldova |
| mc | Monaco |
| mn | Mongolia |
| me | Montenegro |
| ms | Montserrat |
| ma | Morocco |
| mz | Mozambique |
| mm | Myanmar |
| na | Namibia |
| nr | Nauru |
| np | Nepal |
| nl | Netherlands |
| an | Netherlands Antilles |
| nc | New Caledonia |
| nz | New Zealand |
| ni | Nicaragua |
| ne | Niger |
| ng | Nigeria |
| nu | Niue |
| nf | Norfolk Island |
| kp | North Korea |
| vdvn | North Vietnam |
| mp | Northern Mariana Islands |
| no | Norway |
| om | Oman |
| pk | Pakistan |
| pw | Palau |
| xpi | Palestine |
| ps | Palestinian Territory |
| pa | Panama |
| pg | Papua New Guinea |
| py | Paraguay |
| pe | Peru |
| ph | Philippines |
| pl | Poland |
| pt | Portugal |
| pn | Pitcairn |
| pr | Puerto Rico |
| qa | Qatar |
| mk | Republic of Macedonia |
| re | Réunion |
| ro | Romania |
| ru | Russia |
| rw | Rwanda |
| bl | Saint Barthélemy |
| sh | Saint Helena |
| kn | Saint Kitts and Nevis |
| lc | Saint Lucia |
| mf | Saint Martin (French part) |
| pm | Saint Pierre and Miquelon |
| vc | Saint Vincent and the Grenadines |
| ws | Samoa |
| sm | San Marino |
| st | Sao Tome and Principe |
| sa | Saudi Arabia |
| sn | Senegal |
| rs | Serbia |
| csxx | Serbia and Montenegro |
| sc | Seychelles |
| xsi | Siam |
| sl | Sierra Leone |
| sg | Singapore |
| sk | Slovakia |
| si | Slovenia |
| sb | Solomon Islands |
| so | Somalia |
| za | South Africa |
| gs | South Georgia and the South Sandwich Islands |
| kr | South Korea |
| suhh | Soviet Union |
| es | Spain |
| lk | Sri Lanka |
| sd | Sudan |
| sr | Suriname |
| sj | Svalbard and Jan Mayen |
| sz | Swaziland |
| se | Sweden |
| ch | Switzerland |
| sy | Syria |
| tw | Taiwan |
| tj | Tajikistan |
| tz | Tanzania |
| th | Thailand |
| tl | Timor-Leste |
| tg | Togo |
| tk | Tokelau |
| to | Tonga |
| tt | Trinidad and Tobago |
| tn | Tunisia |
| tr | Turkey |
| tm | Turkmenistan |
| tc | Turks and Caicos Islands |
| tv | Tuvalu |
| vi | U.S. Virgin Islands |
| ug | Uganda |
| ua | Ukraine |
| ae | United Arab Emirates |
| gb | United Kingdom |
| us | United States |
| um | United States Minor Outlying Islands |
| uy | Uruguay |
| uz | Uzbekistan |
| vu | Vanuatu |
| ve | Venezuela |
| vn | Vietnam |
| wf | Wallis and Futuna |
| xwg | West Germany |
| eh | Western Sahara |
| ye | Yemen |
| xyu | Yugoslavia |
| zrcd | Zaire |
| zm | Zambia |
| zw | Zimbabwe |